# Validacion con el conjunto de decoys para modelo entrenado sin las reglas de lipinski


## Paso 1. Importación de Librerías
El código comienza importando varias bibliotecas que se utilizan para diversas tareas, como manipulación de datos, visualización y aprendizaje automático. Algunas de las bibliotecas clave incluyen: chembl_webresource_client, rdkit, seaborn, matplotlib, pandas, numpy, scikit-learn, BayesianOptimization, y pickle. Asegúrate de tener estas bibliotecas instaladas en tu entorno de Python.

In [ ]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Draw
from rdkit import Chem
from rdkit.Chem import RDKFingerprint
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.Chem.AllChem import GetHashedTopologicalTorsionFingerprintAsBitVect
from rdkit.Chem import MACCSkeys
from rdkit.DataStructs import ConvertToNumpyArray
from rdkit.Chem import AllChem as Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs
import numpy
import seaborn as sns
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report,precision_score, recall_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report,precision_score, recall_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from bayes_opt import BayesianOptimization
from sklearn import metrics
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics
import pickle
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
import pandas as pd
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from rdkit.Chem import rdMolDescriptors

Seleccion carpeta o localizacion de los datos

In [ ]:
os.chdir('/home/aamaya/BACE1')

### Paso 1: Carga del Modelo Entrenado
El código cambia el directorio de trabajo actual utilizando os.chdir('/home/aamaya/RN').
Luego, carga un modelo previamente entrenado desde un archivo pickle llamado 'modelo_entrenado.pickle' para el modelo entrenado con las 5 reglas de lipinski lo almacena en la variable modelo.

In [ ]:
with open('modelo_entrenadoSIN.pickle', 'rb') as handle:
    modelo = pickle.load(handle)

### Paso 2:  Carga de los Datos de Validación
Los datos de validación, que contienen huellas digitales, se cargan desde un archivo pickle llamado 'decoys_df.pickle' y se almacenan en la variable datos_validacion.



In [ ]:
with open('decoys_df.pickle', 'rb') as handle:
    datos_validacion = pickle.load(handle)

### Paso 3: Conversión de Datos de Validación a DataFrame
El código verifica si datos_validacion es un DataFrame de pandas. Si no lo es, intenta convertirlo en uno.


In [ ]:
print(type(datos_validacion))
if not isinstance(datos_validacion, pd.DataFrame):
    try:
        datos_validacion = pd.DataFrame(datos_validacion)
    except Exception as e:
        print("Error al cargar datos_validacion como DataFrame:", e)
        exit(1)

print(datos_validacion.head())

### Paso 4:Preparación de Datos de Validación
Los datos de validación se procesan para obtener solo las huellas digitales y se convierten en una matriz NumPy X_val.


In [ ]:
fingerprints = datos_validacion.iloc[:, 0].values
X_val = np.array([list(fp.ToBitString()) for fp in fingerprints], dtype=int)

### Paso 5: Realización de Predicciones
Se utilizan el modelo cargado para realizar predicciones en los datos de validación. Las predicciones se almacenan en y_pred, y las probabilidades en y_proba.


In [ ]:
y_pred = modelo.predict(X_val)
y_proba = modelo.predict_proba(X_val)

### Paso 6: Creación de un DataFrame con Resultados
Se crea un DataFrame df que contiene las predicciones (y_pred) y las probabilidades de pertenencia a dos clases (y_proba_0 y y_proba_1).
Este DataFrame se guarda en un archivo CSV llamado 'datos_probabilidadSIN.csv' y en un archivo pickle 'df_probabilidadSIN.pickle'.



In [ ]:
df = pd.DataFrame({'y_pred': y_pred, 'y_proba_0': y_proba[:, 0], 'y_proba_1': y_proba[:, 1]})
df.to_csv('datos_probabilidadSIN.csv', index=False)

### Paso 7: Visualización de Resultados
Se genera un histograma que muestra la distribución de las probabilidades de pertenencia a una de las dos clases. Se utilizan las probabilidades de la clase positiva (y_proba_1).
El histograma se guarda como una imagen llamada 'histogramaSIN.png'. La apariencia del histograma se personaliza mediante ajustes como el rango, el color y la legenda.



In [ ]:
with open('df_probabilidadSIN.pickle', 'wb') as f:
    pickle.dump(df, f)

range_correct = (0, 1)  # Full range from 0 to 1
plt.hist([df[df['y_pred'] == 0]['y_proba_1'], df[df['y_pred'] == 1]['y_proba_1']], bins=20, range=range_correct, alpha=0.5, label=['Inactivo'], color=['red', 'red'], width=0.05)  # Adjust the width as needed (e.g., width=0.1)
y_max = max(df['y_proba_1'].max(), df['y_proba_1'].max())
plt.ylim(0, y_max + 800)  # Adding a buffer of 40 to the y-axis upper limit
plt.legend()
total_data_points = len(df)
plt.title(f"C) Histograma de decoys sin aplicar las reglas de Lipinski\n" 
        f"(n= {total_data_points})")
plt.xlabel("Probabilidad de ser activos")
plt.ylabel('Numero de compuestos')
plt.savefig('histogramaSIN.png') 

### Paso 8: Ejecución del Código
Asegúrate de que el directorio de trabajo actual tenga acceso a los archivos de modelo y datos de validación. Ejecuta el código, que cargará el modelo, realizará predicciones en los datos de validación y generará un histograma. Los resultados se guardarán en archivos CSV y pickle para su posterior análisis. La imagen del histograma se guardará como 'histogramaSIN.png'.



Script elaborado por Nelson Alejandro Amaya Orozco, como trabajo de grado, para el grupo de investigacion RamirezLAB